In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

def extract_kill_info(file):
    # Load and parse the XML file
    tree = ET.parse(file)  # Adjust the path if your file is elsewhere
    root = tree.getroot()
    
    # Prepare list to collect extracted data
    data = []
    
    # Traverse each skill
    for skill in root.findall(".//imgdir[@name='skill']/imgdir"):
        skill_id = skill.attrib.get('name')  # Skill ID like 1100000
        for level in skill.findall("imgdir[@name='level']/imgdir"):
            level_id = level.attrib.get('name')  # Level like 1, 2, 3, etc.
            row = {
                'skill_id': skill_id,
                'level': level_id,
                'mobCount': None,
                'lt': None,
                'rb': None,
                'damage': None,
                'mpCon': None,
                'hpCon': None
            }
            # Extract relevant fields
            for child in level:
                tag_name = child.attrib['name']
                if child.tag == 'int' and tag_name in row:
                    row[tag_name] = child.attrib['value']
                elif child.tag == 'vector' and tag_name in ['lt', 'rb']:
                    coords = (child.attrib['x'], child.attrib['y'])
                    row[tag_name] = str(coords)
            data.append(row)
    
    # Convert to a pandas DataFrame
    df = pd.DataFrame(data)
    
    # Display or export as needed
    # print(df)
    # df.to_excel(file[:-8] + ".xlsx", index=False)  # Optional: save to Excel
    print(f'Done for {file[:-8]}')
    return df

path = r"D:\MerogieMS\wz\Skill.wz\\"
directory = os.fsencode(path)
df_list = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xml"): 
        df_list.append(extract_kill_info(path + filename))
    else:
        continue
pd.concat(df_list).to_excel('Skill.xlsx')